<a href="https://colab.research.google.com/github/eclipseeyo/practiceML/blob/main/LogReg_%D0%B4%D0%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [1]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [35]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [29]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [30]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [31]:
# ваш код здесь
data.describe(include="object")

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [36]:
data = data.drop(['Education','Occupation','Commute Distance','Region'], axis=1)
data.describe(include="object")

,Marital Status,Gender,Home Owner,Purchased Bike
count,1000,1000,1000,1000
unique,2,2,2,2
top,Married,Male,Yes,No
freq,539,509,685,519


In [37]:
# ваш код здесь
data['Marital Status'] = data['Marital Status'].apply(lambda x: 1 if x == 'Married' else 0)
data['Gender'] = data['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
data['Home Owner'] = data['Home Owner'].apply(lambda x: 1 if x == 'Yes' else 0)
data['Purchased Bike'] = data['Purchased Bike'].apply(lambda x: 1 if x == 'No' else 0)

In [39]:
data.head(10)

,ID,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,12496,1,0,40000,1,1,0,42,1
1,24107,1,1,30000,3,1,1,43,1
2,14177,1,1,80000,5,0,2,60,1
3,24381,0,1,70000,0,1,1,41,0
4,25597,0,1,30000,0,0,0,36,0
5,13507,1,0,10000,2,1,0,50,1
6,27974,0,1,160000,2,1,4,33,0
7,19364,1,1,40000,1,1,0,43,0
8,22155,1,1,20000,2,1,2,58,1
9,19280,1,1,20000,2,1,1,48,0


Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [ ]:
# ваш код здесь

Удалите столбец `ID`, так как он по сути нечисловой.

In [40]:
data = data.drop('ID', axis=1)

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [41]:
X = data.drop('Purchased Bike',axis=1)
y = data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [63]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [64]:
Xtrain.shape

(700, 7)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [65]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

Xtrain_scaled = scaler.fit_transform(Xtrain)

Xtest_scaled = scaler.transform(Xtest)

Xtrain = pd.DataFrame(Xtrain_scaled, columns=Xtrain.columns, index=Xtrain.index)
Xtest = pd.DataFrame(Xtest_scaled, columns=Xtest.columns, index=Xtest.index)

Теперь обучите логистическую регрессию на тренировочных данных

In [66]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtrain,ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [67]:
train_prediction = lr.predict(Xtrain)
prediction = lr.predict(Xtest)

Оцените значение accuracy на трейне и на тесте.

In [68]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(ytrain, train_prediction)
test_acc = accuracy_score(ytest, prediction)
print(train_acc)
print(test_acc)

0.6342857142857142
0.5766666666666667


Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [69]:
from sklearn.preprocessing import PolynomialFeatures

polydf = PolynomialFeatures(degree=2, include_bias=True)

Xtrain_scaled = polydf.fit_transform(Xtrain)
Xtest_scaled = polydf.transform(Xtest)

feature_names = polydf.get_feature_names_out(input_features=Xtrain.columns)

Xtrain = pd.DataFrame(Xtrain_scaled, columns=feature_names, index=Xtrain.index)
Xtest = pd.DataFrame(Xtest_scaled, columns=feature_names, index=Xtest.index)


In [70]:
Xtrain.shape

(700, 36)

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [72]:
# ваш код здесь
lr.fit(Xtrain, ytrain)

train_prediction = lr.predict(Xtrain)
prediction = lr.predict(Xtest)

train_acc_new = accuracy_score(ytrain, train_prediction)
test_acc_new = accuracy_score(ytest, prediction)
print(train_acc)
print(test_acc)
test_acc_new - 0.5766666666666667

0.6857142857142857
0.6233333333333333


0.046666666666666634

**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [73]:
from sklearn.metrics import recall_score

ypred = lr.predict(Xtest)

accuracy = accuracy_score(ytest, ypred)
recall = recall_score(ytest, ypred)

In [76]:
accuracy, recall

(0.6233333333333333, 0.6959459459459459)

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [89]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

lr.fit(XtrainS,ytrainS)
probas = lr.predict_proba(Xval)[:, 1]

array([0.82373351, 0.25244547, 0.45524041, 0.7241238 , 0.42489463,
       0.68056497, 0.87697861, 0.48581293, 0.43960499, 0.67228894])

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [90]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    y_pred = (probas >= thr).astype(int)

    current_recall = recall_score(yval, y_pred)
    current_acc = accuracy_score(yval, y_pred)

    if current_acc >= 0.6 and current_recall > RecMax:
        RecMax = current_recall
        BestThr = thr
        BestAcc = current_acc

print(BestThr, RecMax, BestAcc)

0.42 0.8518518518518519 0.6142857142857143


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [94]:
lr.fit(Xtrain, ytrain)

test_probas = lr.predict_proba(Xtest)[:, 1]

y_pred_test = (test_probas >= BestThr).astype(int)

test_recall = recall_score(ytest, y_pred_test)
test_accuracy = accuracy_score(ytest, y_pred_test)
test_recall, test_accuracy

(0.8040540540540541, 0.5666666666666667)

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.